In [1]:
import numpy as np
import cv2
from PIL import Image
import open3d as o3d
import json
import os
from tqdm import tqdm
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib import rcParams
from shapely.geometry import MultiPoint, box

# from pyquaternion.quaternion import Quaternion
from pyquaternion import Quaternion
import os.path as osp
from nuscenes import NuScenes

# Utils for Lidar and Radar
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.scripts.export_2d_annotations_as_json import get_2d_boxes, post_process_coords
from utils import *

from typing import Tuple, List, Dict, Union

%matplotlib inline

In [2]:
#nusc = NuScenes(version='v1.0-mini', dataroot='/home/gus/Documents/AI_S/nuScenes/v1_0-mini', verbose=True)
nusc = NuScenes(version='v1.0-mini', dataroot='/home/sherlock/Documents/nuScenes/data', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.5 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


## Loop to create the features and targets file

In [3]:
# path with all the sample files
samples_path =  os.getcwd() + '/dataset/samples/'

# path to retrieve point clous features
features_path =  os.getcwd() + '/dataset/point_features_path/'

# Matrix to store the features' vectors
features_matrix = []
# Dictionary of targets
targets_dic = {'x_pos': [], 'y_pos': [], 'z_pos': [], 'width': [], 'lenght': [], 'height': [], 'yaw': []}

tokens_dic = {'sample_token': [], 'camera_token': [], 'category': []}

# extract all files in path
for root, dirs, files in os.walk(samples_path):
    
    # select each sample file
    for file_ in tqdm(files, desc = "Sample files"):    
        
        # load the current sample file
        data_json = load_file(samples_path + file_)
                
        # process each instance
        for idx in range(len(data_json['instance'])):
            
            # Create instance metadata
            annotation_metadata = data_json['instance'][idx]

            # Retrieve instance's features
            features_vec = parse_features_to_numpy(annotation_metadata['point_features_path'])
            
            # Store the current feature vector in the features' matrix
            features_matrix.append(list(features_vec))
            
            # Extract class targets
            pose = annotation_metadata['position_coord']
            dims = annotation_metadata['wlh_values']
            yaw = annotation_metadata['orientation_value']
            label = annotation_metadata['category']
            
            # Save X, Y and Z pose
            targets_dic['x_pos'].append(pose[0])
            targets_dic['y_pos'].append(pose[1])
            targets_dic['z_pos'].append(pose[2])
            
            # Store target size
            targets_dic['width'].append(dims[0])
            targets_dic['lenght'].append(dims[1])
            targets_dic['height'].append(dims[2])
            
            # Store target orientation
            targets_dic['yaw'].append(yaw)
            
            # Store sample, camera and annotation token
            tokens_dic['sample_token'].append(annotation_metadata['sample_token'])
            tokens_dic['camera_token'].append(annotation_metadata['camera_token'])
            tokens_dic['category'].append(annotation_metadata['category'])

            
# Parse features as a numpy array
features_matrix = np.array(features_matrix)

# Create a dataframe of features
df_features = pd.DataFrame(data = features_matrix)
# Create a dataframe of targets
df_targets = pd.DataFrame.from_dict(targets_dic)
# Create a dataframe of tokens
df_tokens = pd.DataFrame.from_dict(tokens_dic)

# Concat both dataframes horizontally to create the final dataframe
df_detection = pd.concat([df_features, df_targets, df_tokens], axis=1)
df_detection.head()

df_detection.to_csv('dataset/3d_obj_detection.csv')

Sample files: 100%|██████████| 404/404 [00:02<00:00, 192.21it/s]


## Loading CSV and parsing info as a numpy array

In [20]:
# Load dataframe form CSV file
df_detection_csv = pd.read_csv('dataset/3d_obj_detection.csv')
# features: 1 - 640 // targets: 641 - 648

print('The Dataframe shape is {}'.format(df_detection_csv.shape)) 
print('The targets keys are',df_detection_csv.keys()[641:648])

# Parse features and targets to numpy array
features = df_detection_csv.loc[:,'0':'639'].to_numpy()
targets = df_detection_csv.loc[:,'x_pos':'yaw'].to_numpy()

print('The Features shape is {}'.format(features.shape))
print('The Targets shape is {}'.format(targets.shape))

The Dataframe shape is (1415, 648)
The target keys are Index(['x_pos', 'y_pos', 'z_pos', 'width', 'lenght', 'height', 'yaw'], dtype='object')
The Features shape is (1415, 640)
The targets shape is (1415, 7)


## Splitting and scaling the data

In [69]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import RobustScaler 
from sklearn.preprocessing import Normalizer 

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=1)

print('Train set features size: \n {}'.format(X_train.shape))
print('Test set featires size: \n {}'.format(X_test.shape))
print('Train set targets size: \n {}'.format(y_train.shape))
print('Test set targets size: \n {}'.format(y_test.shape))

Train set features size: 
 (1132, 640)
Test set featires size: 
 (283, 640)
Train set targets size: 
 (1132, 7)
Test set targets size: 
 (283, 7)


## Transformation

In [98]:
scaler = MinMaxScaler()
scaler.fit(X_train) 
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Multi output regressor Ridge 

In [106]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

clf = MultiOutputRegressor(Ridge(random_state = 123)).fit(X_train_scaled, y_train)

In [107]:
print('Training set score is: {:.2f}'.format(clf.score(X_train_scaled, y_train)))
print('Test set score is: {:.2f}'.format(clf.score(X_test_scaled, y_test)))

Training set score is: 0.61
Test set score is: 0.18


In [108]:
ridge_pred_train = clf.predict(X_train_scaled)
print('Train prediction: ', ridge_pred_train[50])
print("Train Targets: ", y_train[50] )
# ['x_pos', 'y_pos', 'z_pos', 'width', 'lenght', 'height', 'yaw']

Train prediction:  [  1.46711326  -0.98160548  -0.39672363   2.64321882   7.41612475
   3.20406077 -77.47530059]
Train Targets:  [ -2.99464996  -2.70744051  -0.48460726   2.749        8.702
   3.078      -96.66277978]


In [110]:
ridge_pred_test = clf.predict(X_test_scaled)
print('Test prediction: ', ridge_pred_test[50])
print("Test Targets: ", y_test[50] )
# ['x_pos', 'y_pos', 'z_pos', 'width', 'lenght', 'height', 'yaw']

Test prediction:  [  4.09826282   5.4450462    0.03370223   2.75658005  10.09993186
   3.39902382 -23.9578553 ]
Test Targets:  [ 4.74473776  9.21747484  0.26964428  2.764       9.48        3.374
 88.13149184]


## Multi output regressor SVR¶

In [103]:
from sklearn import svm

clf = MultiOutputRegressor(svm.SVR(kernel='linear')).fit(X_train_scaled, y_train)

print('Training set score is: {:.2f}'.format(clf.score(X_train_scaled, y_train)))
print('Test set score is: {:.2f}'.format(clf.score(X_test_scaled, y_test)))

Training set score is: 0.46
Test set score is: 0.16


In [104]:
svm_pred_train = clf.predict(X_train_scaled)
print('Train prediction: ', svm_pred_train[60])
print("Train Targets: ", y_train[60] )
# ['x_pos', 'y_pos', 'z_pos', 'width', 'lenght', 'height', 'yaw']

Train prediction:  [  8.57749938  -0.55795344  -0.96600857   2.03505087   4.41141789
   1.63736505 -63.0013467 ]
Train Targets:  [ 14.25424176  -0.45802208  -1.01023042   1.935        4.494
   1.537      176.6700548 ]


In [105]:
svm_pred_test = clf.predict(X_test_scaled)
print('Test prediction: ', svm_pred_test[50])
print("Test Targets: ", y_test[50] )
# ['x_pos', 'y_pos', 'z_pos', 'width', 'lenght', 'height', 'yaw']

Test prediction:  [ 7.27030335  3.88958425 -0.103656    2.73236528  8.82365857  2.89078306
 82.57343486]
Test Targets:  [ 4.74473776  9.21747484  0.26964428  2.764       9.48        3.374
 88.13149184]


## MLP Regressor

In [101]:
from sklearn.neural_network import MLPRegressor

mlp_reg = MLPRegressor(hidden_layer_sizes=1200,random_state=1, max_iter=800).fit(X_train_scaled, y_train)
mlp_reg.n_layers_

4

In [102]:
mlp_pred_test = mlp_reg.predict(X_test_scaled)
print('Test prediction: ', mlp_pred_test[50])
print("Test Targets: ", y_test[50] )
# ['x_pos', 'y_pos', 'z_pos', 'width', 'lenght', 'height', 'yaw']

Test prediction:  [ -2.15200853  -1.80625099  -0.82408711   1.88459698   5.26014426
   1.8849862  113.8675372 ]
Test Targets:  [ 4.74473776  9.21747484  0.26964428  2.764       9.48        3.374
 88.13149184]


---